# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [29]:
!pip install imageio

In [1]:
import numpy as np
import os
import imageio
from PIL import Image
#resize
import datetime
import os

We set the random seed so that the results don't vary drastically.

In [2]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.random.set_seed(30)

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [3]:
train_doc = np.random.permutation(open('Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('Project_data/val.csv').readlines())
batch_size = 40

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [4]:
def crop_img(image, crop_fraction):
    """
    Crop the image by a certain fraction of its width, maintaining the center.
    The crop_fraction should be a float representing the fraction of the width to crop.
    The image's RGB channels will be preserved.
    
    Parameters:
    - image: numpy array of shape (height, width, channels), representing the image.
    - crop_fraction: float, the fraction of the image's width to crop (0.10 means cropping 10% from both sides).
    
    Returns:
    - cropped_image: numpy array of the cropped image.
    """
    # Ensure the image has 3 channels (RGB)
    if image.shape[2] != 3:
        raise ValueError("Image should have 3 channels (RGB).")

    # Get the height, width, and channels of the image
    h, w, c = image.shape

    # Calculate the width to crop from each side
    crop_w = int(crop_fraction * w)

    # Determine the start and end indices for cropping
    start_w = crop_w // 2
    end_w = w - crop_w // 2

    # Crop the image, preserving all RGB channels
    cropped_image = image[:, start_w:end_w, :]

    return cropped_image


In [5]:
def generator(source_path, folder_list, batch_size):
    print('Source path = ', source_path, '; batch size =', batch_size)
    img_idx = list(range(30))  # We will use 30 frames per video (adjustable)
    x, y, z = len(img_idx), 160, 160  # x is the number of images, y and z are the target dimensions

    while True:
        t = np.random.permutation(folder_list)  # Shuffle the list of folders (videos)
        num_batches = len(folder_list) // batch_size  # Calculate number of full batches
        print('num_batches = ', num_batches)

        for batch in range(num_batches):  # Iterate over the number of batches
            batch_data = np.zeros((batch_size, x, y, z, 3))  # Initialize batch data (40 samples, 30 frames, 160x160, 3 channels)
            batch_labels = np.zeros((batch_size, 5))  # Initialize batch labels (one-hot encoding for 5 classes)
            
            for folder in range(batch_size):  # Iterate over the batch size (40 samples)
                folder_path = source_path + '/' + t[folder + (batch * batch_size)].split(';')[0]
                imgs = os.listdir(folder_path)  # Read all image files in the folder

                for idx, item in enumerate(img_idx):  # Iterate over the frames/images (using img_idx)
                    img_path = os.path.join(folder_path, imgs[item])  # Get the image path
                    image = imageio.imread(img_path).astype(np.float32)  # Read the image using imageio
                    
                    # Crop and resize images to ensure uniformity in shape
                    image = crop_img(image, 0.10)  # Crop 10% from both sides
                    
                    # Resize the image to 160x160 using PIL 
                    image = Image.fromarray(image.astype(np.uint8))  # Convert numpy array to PIL image
                    image = image.resize((y, z), Image.Resampling.LANCZOS)  # Resize using Pillow
                    image = np.array(image).astype(np.float32)  # Convert back to numpy array
                    
                    # Normalize the image by mean normalization (subtract mean and divide by standard deviation)
                    image = (image - np.mean(image)) / np.std(image)  # Mean normalization

                    # Load the image into the 3 RGB channels
                    # Ensure we only store each colour channel separately
                    batch_data[folder, idx, :, :, 0] = image[:, :, 0]  # Red channel
                    batch_data[folder, idx, :, :, 1] = image[:, :, 1]  # Green channel
                    batch_data[folder, idx, :, :, 2] = image[:, :, 2]  # Blue channel

                # Set the one-hot encoding for the label (class)
                class_idx = int(t[folder + (batch * batch_size)].strip().split(';')[2])  # Get the class index from the CSV file
                batch_labels[folder, class_idx] = 1  # One-hot encoding for the class label

            yield batch_data, batch_labels  # Yield the batch of data and labels

        # Handle remaining data points (if not a perfect multiple of batch_size)
        if len(folder_list) % batch_size != 0:
            remaining_samples = len(folder_list) % batch_size
            batch_data_remaining = np.zeros((remaining_samples, x, y, z, 3))  # Remaining batch data
            batch_labels_remaining = np.zeros((remaining_samples, 5))  # Remaining batch labels

            for folder in range(remaining_samples):  # Process the remaining samples
                folder_path = source_path + '/' + t[folder + (num_batches * batch_size)].split(';')[0]
                imgs = os.listdir(folder_path)  # Read all image files in the folder

                for idx, item in enumerate(img_idx):  # Iterate over frames/images of a folder
                    img_path = os.path.join(folder_path, imgs[item])  # Get the image path
                    image = imageio.imread(img_path).astype(np.float32)  # Read the image

                    # Crop and resize the images to ensure uniform shape
                    image = crop_img(image, 0.10)  # Crop 10% from both sides
                    
                    # Resize the image to 160x160 using PIL
                    image = Image.fromarray(image.astype(np.uint8))  # Convert numpy array to PIL image
                    image = image.resize((y, z), Image.Resampling.LANCZOS)  # Resize using Pillow
                    image = np.array(image).astype(np.float32)  # Convert back to numpy array
                    
                    # Normalize the image
                    image = (image - np.mean(image)) / np.std(image)  # Mean normalization

                    # Load the image into the 3 RGB channels
                    # Ensure we only store each colour channel separately
                    batch_data[folder, idx, :, :, 0] = image[:, :, 0]  # Red channel
                    batch_data[folder, idx, :, :, 1] = image[:, :, 1]  # Green channel
                    batch_data[folder, idx, :, :, 2] = image[:, :, 2]  # Blue channel

                # Set the one-hot encoding for the class
                class_idx = int(t[folder + (num_batches * batch_size)].strip().split(';')[2])
                batch_labels_remaining[folder, class_idx] = 1  # One-hot encoding for the class label

            yield batch_data_remaining, batch_labels_remaining  # Yield the final batch with the remaining data


Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [6]:
curr_dt_time = datetime.datetime.now()
train_path = 'Project_data/train'
val_path = 'Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 30
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 30


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

#### Experiment 1

Conv 3D Model with 30 epochs, 40 batch size

Without dropouts in Conv layer and with batch normalization

Input image size 160x160 , adam optimiser with learning rate 0.0001, 30 images as input out of 30

In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv3D, MaxPooling3D, Flatten, BatchNormalization, Activation
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras import optimizers

# Initialize the model
model = Sequential()

# Add 3D Convolutional Layers and Max Pooling Layers
# Input shape is (30, 160, 160, 3) -> (number of frames, height, width, channels)

# First Conv3D layer
model.add(Conv3D(16, (3, 3, 3), padding='same', input_shape=(30, 160, 160, 3)))  
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

# Second Conv3D layer
model.add(Conv3D(32, (3, 3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

# Third Conv3D layer
model.add(Conv3D(64, (3, 3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

# Flatten the output of Conv3D layers before passing to fully connected layers
model.add(Flatten())

# Dense Layer for Classification
model.add(Dense(128))
model.add(Activation('relu'))

# Output Layer (5 classes for classification)
model.add(Dense(5, activation='softmax'))  # Assuming there are 5 classes for classification

/Users/rajashekar/Downloads/Softwares/Anaconda/anaconda3/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [8]:
# Compile the model with an optimizer
optimiser = optimizers.Adam(learning_rate=0.0001)  # You can experiment with the learning rate
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

# Print the model summary to check the total number of parameters
print(model.summary())

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv3d (Conv3D)                 │ (None, 30, 160, 160,   │         1,312 │
│                                 │ 16)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 30, 160, 160,   │            64 │
│ (BatchNormalization)            │ 16)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 30, 160, 160,   │             0 │
│                                 │ 16)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d (MaxPooling3D)    │ (None, 15, 80, 80, 16) │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d_1 (Conv3D)               │ (None, 15, 80, 80, 32) │        13,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 15, 80, 80, 32) │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 15, 80, 80, 32) │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d_1 (MaxPooling3D)  │ (None, 7, 40, 40, 32)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d_2 (Conv3D)               │ (None, 7, 40, 40, 64)  │        55,360 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 7, 40, 40, 64)  │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_2 (Activation)       │ (None, 7, 40, 40, 64)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d_2 (MaxPooling3D)  │ (None, 3, 20, 20, 64)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 76800)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     9,830,528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_3 (Activation)       │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 5)              │           645 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9,902,149 (37.77 MB)

 Trainable params: 9,901,925 (37.77 MB)

 Non-trainable params: 224 (896.00 B)

None


Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [9]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [10]:
from keras.callbacks import ReduceLROnPlateau

model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
# Save the model in .keras format during training
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.weights.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=0, 
                             save_best_only=False, save_weights_only=True, mode='auto',  save_freq='epoch')

# Define the ReduceLROnPlateau callback
LR = ReduceLROnPlateau(monitor='val_loss', 
                       factor=0.2, 
                       patience=5, 
                       verbose=1, 
                       min_lr=0.0001)

callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [11]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

print('Steps per epoch:', steps_per_epoch)
print('Validation steps:', validation_steps)

Steps per epoch: 17
Validation steps: 3


Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [13]:
model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, initial_epoch=0)

Source path =  Project_data/train ; batch size = 40
num_batches =  16


/var/folders/th/mw8md0ks7rsdgy45pvbb9cgw0000gn/T/ipykernel_10673/1150823310.py:21: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(img_path).astype(np.float32)  # Read the image using imageio


Epoch 1/30
15/17 ━━━━━━━━━━━━━━━━━━━━ 59s 30s/step - categorical_accuracy: 0.2098 - loss: 6.0074 

/var/folders/th/mw8md0ks7rsdgy45pvbb9cgw0000gn/T/ipykernel_10673/1150823310.py:58: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(img_path).astype(np.float32)  # Read the image


16/17 ━━━━━━━━━━━━━━━━━━━━ 29s 29s/step - categorical_accuracy: 0.2136 - loss: 5.8725num_batches =  16
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 28s/step - categorical_accuracy: 0.2166 - loss: 5.7489 Source path =  Project_data/val ; batch size = 40
num_batches =  2
num_batches =  2

Epoch 1: saving model to model_init_2024-12-2608_18_45.693183/model-00001-3.77187-0.26546-1.53121-0.31000.keras
17/17 ━━━━━━━━━━━━━━━━━━━━ 504s 30s/step - categorical_accuracy: 0.2193 - loss: 5.6391 - val_categorical_accuracy: 0.3100 - val_loss: 1.5312 - learning_rate: 1.0000e-04
Epoch 2/30
16/17 ━━━━━━━━━━━━━━━━━━━━ 57s 57s/step - categorical_accuracy: 0.5335 - loss: 1.1130 num_batches =  16
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 54s/step - categorical_accuracy: 0.5356 - loss: 1.1121 num_batches =  2

Epoch 2: saving model to model_init_2024-12-2608_18_45.693183/model-00002-1.09797-0.56863-1.48434-0.32500.keras
17/17 ━━━━━━━━━━━━━━━━━━━━ 954s 55s/step - categorical_accuracy: 0.5374 - loss: 1.1113 - val_categorical_accuracy: 0

17/17 ━━━━━━━━━━━━━━━━━━━━ 509s 30s/step - categorical_accuracy: 0.9970 - loss: 0.0560 - val_categorical_accuracy: 0.4917 - val_loss: 1.3587 - learning_rate: 1.0000e-04
Epoch 15/30
16/17 ━━━━━━━━━━━━━━━━━━━━ 29s 30s/step - categorical_accuracy: 1.0000 - loss: 0.0447num_batches =  16
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 29s/step - categorical_accuracy: 0.9987 - loss: 0.0479 num_batches =  2

Epoch 15: saving model to model_init_2024-12-2608_18_45.693183/model-00015-0.09852-0.97738-1.48156-0.49167.keras
17/17 ━━━━━━━━━━━━━━━━━━━━ 502s 29s/step - categorical_accuracy: 0.9975 - loss: 0.0507 - val_categorical_accuracy: 0.4917 - val_loss: 1.4816 - learning_rate: 1.0000e-04
Epoch 16/30
16/17 ━━━━━━━━━━━━━━━━━━━━ 30s 30s/step - categorical_accuracy: 1.0000 - loss: 0.0394 num_batches =  16
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 29s/step - categorical_accuracy: 0.9983 - loss: 0.0426 num_batches =  2

Epoch 16: saving model to model_init_2024-12-2608_18_45.693183/model-00016-0.09430-0.97134-1.48382-0.51667.keras

So we have got the training accuarcy of 99 % and validation accuracy of 47 % in the base model

#### Experiment - 2
Conv 3D Model with 15 epochs, 25 batch size

Without dropouts in Conv layer and with batch normalization

Input image size 120x120 , adam optimiser with learning rate 0.0001, 18 images as input out of 30

In [15]:
def generator_2(source_path, folder_list, batch_size):
    print('Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [0,1,2,4,6,8,10,12,14,16,18,20,22,24,26,27,28,29]
    #list(range(30))  # We will use 30 frames per video (adjustable)
    x, y, z = len(img_idx), 120, 120  # x is the number of images, y and z are the target dimensions

    while True:
        t = np.random.permutation(folder_list)  # Shuffle the list of folders (videos)
        num_batches = len(folder_list) // batch_size  # Calculate number of full batches
        print('num_batches = ', num_batches)

        for batch in range(num_batches):  # Iterate over the number of batches
            batch_data = np.zeros((batch_size, x, y, z, 3))  # Initialize batch data (40 samples, 30 frames, 160x160, 3 channels)
            batch_labels = np.zeros((batch_size, 5))  # Initialize batch labels (one-hot encoding for 5 classes)
            
            for folder in range(batch_size):  # Iterate over the batch size (40 samples)
                folder_path = source_path + '/' + t[folder + (batch * batch_size)].split(';')[0]
                imgs = os.listdir(folder_path)  # Read all image files in the folder

                for idx, item in enumerate(img_idx):  # Iterate over the frames/images (using img_idx)
                    img_path = os.path.join(folder_path, imgs[item])  # Get the image path
                    image = imageio.imread(img_path).astype(np.float32)  # Read the image using imageio
                    
                    # Crop and resize images to ensure uniformity in shape
                    image = crop_img(image, 0.10)  # Crop 10% from both sides
                    
                    # Resize the image to 160x160 using PIL 
                    image = Image.fromarray(image.astype(np.uint8))  # Convert numpy array to PIL image
                    image = image.resize((y, z), Image.Resampling.LANCZOS)  # Resize using Pillow
                    image = np.array(image).astype(np.float32)  # Convert back to numpy array
                    
                    # Normalize the image by mean normalization (subtract mean and divide by standard deviation)
                    image = (image - np.mean(image)) / np.std(image)  # Mean normalization

                    # Load the image into the 3 RGB channels
                    # Ensure we only store each colour channel separately
                    batch_data[folder, idx, :, :, 0] = image[:, :, 0]  # Red channel
                    batch_data[folder, idx, :, :, 1] = image[:, :, 1]  # Green channel
                    batch_data[folder, idx, :, :, 2] = image[:, :, 2]  # Blue channel

                # Set the one-hot encoding for the label (class)
                class_idx = int(t[folder + (batch * batch_size)].strip().split(';')[2])  # Get the class index from the CSV file
                batch_labels[folder, class_idx] = 1  # One-hot encoding for the class label

            yield batch_data, batch_labels  # Yield the batch of data and labels

        # Handle remaining data points (if not a perfect multiple of batch_size)
        if len(folder_list) % batch_size != 0:
            remaining_samples = len(folder_list) % batch_size
            batch_data_remaining = np.zeros((remaining_samples, x, y, z, 3))  # Remaining batch data
            batch_labels_remaining = np.zeros((remaining_samples, 5))  # Remaining batch labels

            for folder in range(remaining_samples):  # Process the remaining samples
                folder_path = source_path + '/' + t[folder + (num_batches * batch_size)].split(';')[0]
                imgs = os.listdir(folder_path)  # Read all image files in the folder

                for idx, item in enumerate(img_idx):  # Iterate over frames/images of a folder
                    img_path = os.path.join(folder_path, imgs[item])  # Get the image path
                    image = imageio.imread(img_path).astype(np.float32)  # Read the image

                    # Crop and resize the images to ensure uniform shape
                    image = crop_img(image, 0.10)  # Crop 10% from both sides
                    
                    # Resize the image to 160x160 using PIL
                    image = Image.fromarray(image.astype(np.uint8))  # Convert numpy array to PIL image
                    image = image.resize((y, z), Image.Resampling.LANCZOS)  # Resize using Pillow
                    image = np.array(image).astype(np.float32)  # Convert back to numpy array
                    
                    # Normalize the image
                    image = (image - np.mean(image)) / np.std(image)  # Mean normalization

                    # Load the image into the 3 RGB channels
                    # Ensure we only store each colour channel separately
                    batch_data[folder, idx, :, :, 0] = image[:, :, 0]  # Red channel
                    batch_data[folder, idx, :, :, 1] = image[:, :, 1]  # Green channel
                    batch_data[folder, idx, :, :, 2] = image[:, :, 2]  # Blue channel

                # Set the one-hot encoding for the class
                class_idx = int(t[folder + (num_batches * batch_size)].strip().split(';')[2])
                batch_labels_remaining[folder, class_idx] = 1  # One-hot encoding for the class label

            yield batch_data_remaining, batch_labels_remaining  # Yield the final batch with the remaining data


In [16]:
train_doc = np.random.permutation(open('Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('Project_data/val.csv').readlines())
batch_size = 25
num_epochs = 15
print ('# epochs =', num_epochs)

# epochs = 15


In [17]:
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers

In [18]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv3D, MaxPooling3D, BatchNormalization, Activation, Dropout, GlobalAveragePooling3D, Dense
from tensorflow.keras import regularizers

# Hyperparameters
dropout_rate = 0.5
l2_reg = 0.01
num_classes = 5

# Initialize the model
model = Sequential()

# First Conv3D layer
model.add(Conv3D(
    16, 
    (3, 3, 3), 
    padding='same', 
    input_shape=(18, 120, 120, 3), 
    kernel_regularizer=regularizers.l2(l2_reg)
))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))


# Second Conv3D layer
model.add(Conv3D(
    32, 
    (3, 3, 3), 
    padding='same', 
    kernel_regularizer=regularizers.l2(l2_reg)
))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))


# Third Conv3D layer
model.add(Conv3D(
    64, 
    (3, 3, 3), 
    padding='same', 
    kernel_regularizer=regularizers.l2(l2_reg)
))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))


# Global Average Pooling and Fully Connected Layers
model.add(GlobalAveragePooling3D())
model.add(Dropout(dropout_rate))
model.add(Dense(128, activation='relu', kernel_regularizer=regularizers.l2(l2_reg)))


# Output Layer
model.add(Dense(num_classes, activation='softmax'))


In [19]:
# Compile the model with an optimizer
optimiser = optimizers.Adam(learning_rate=0.0001)  # You can experiment with the learning rate
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

# Print the model summary to check the total number of parameters
print(model.summary())

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv3d_3 (Conv3D)               │ (None, 18, 120, 120,   │         1,312 │
│                                 │ 16)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 18, 120, 120,   │            64 │
│ (BatchNormalization)            │ 16)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_4 (Activation)       │ (None, 18, 120, 120,   │             0 │
│                                 │ 16)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d_3 (MaxPooling3D)  │ (None, 9, 60, 60, 16)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d_4 (Conv3D)               │ (None, 9, 60, 60, 32)  │        13,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 9, 60, 60, 32)  │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_5 (Activation)       │ (None, 9, 60, 60, 32)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d_4 (MaxPooling3D)  │ (None, 4, 30, 30, 32)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d_5 (Conv3D)               │ (None, 4, 30, 30, 64)  │        55,360 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 4, 30, 30, 64)  │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_6 (Activation)       │ (None, 4, 30, 30, 64)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d_5 (MaxPooling3D)  │ (None, 2, 15, 15, 64)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling3d        │ (None, 64)             │             0 │
│ (GlobalAveragePooling3D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 5)              │           645 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 79,941 (312.27 KB)

 Trainable params: 79,717 (311.39 KB)

 Non-trainable params: 224 (896.00 B)

None


In [20]:
train_generator = generator_2(train_path, train_doc, batch_size)
val_generator = generator_2(val_path, val_doc, batch_size)

In [21]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

print('Steps per epoch:', steps_per_epoch)
print('Validation steps:', validation_steps)

Steps per epoch: 27
Validation steps: 4


In [21]:
model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, initial_epoch=0)

Source path =  Project_data/train ; batch size = 25
num_batches =  26


/var/folders/th/mw8md0ks7rsdgy45pvbb9cgw0000gn/T/ipykernel_31353/4273982213.py:22: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(img_path).astype(np.float32)  # Read the image using imageio


Epoch 1/15
25/27 ━━━━━━━━━━━━━━━━━━━━ 10s 5s/step - categorical_accuracy: 0.2002 - loss: 3.4224

/var/folders/th/mw8md0ks7rsdgy45pvbb9cgw0000gn/T/ipykernel_31353/4273982213.py:59: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(img_path).astype(np.float32)  # Read the image


26/27 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step - categorical_accuracy: 0.2001 - loss: 3.4187 num_batches =  26
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - categorical_accuracy: 0.1999 - loss: 3.4151Source path =  Project_data/val ; batch size = 25
num_batches =  4
num_batches =  4

Epoch 1: saving model to model_init_2024-12-2619_34_37.119648/model-00001-3.32135-0.19457-3.08584-0.22000.keras
27/27 ━━━━━━━━━━━━━━━━━━━━ 149s 6s/step - categorical_accuracy: 0.1997 - loss: 3.4117 - val_categorical_accuracy: 0.2200 - val_loss: 3.0858 - learning_rate: 1.0000e-04
Epoch 2/15
26/27 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step - categorical_accuracy: 0.2602 - loss: 3.1374 num_batches =  26
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - categorical_accuracy: 0.2600 - loss: 3.1374num_batches =  4

Epoch 2: saving model to model_init_2024-12-2619_34_37.119648/model-00002-3.13861-0.25641-3.05300-0.24000.keras
27/27 ━━━━━━━━━━━━━━━━━━━━ 146s 5s/step - categorical_accuracy: 0.2599 - loss: 3.1375 - val_categorical_accuracy: 0.2400 - v

27/27 ━━━━━━━━━━━━━━━━━━━━ 148s 5s/step - categorical_accuracy: 0.4595 - loss: 2.5573 - val_categorical_accuracy: 0.4900 - val_loss: 2.5157 - learning_rate: 1.0000e-04
Epoch 15/15
26/27 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step - categorical_accuracy: 0.4580 - loss: 2.5370 num_batches =  26
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - categorical_accuracy: 0.4578 - loss: 2.5377num_batches =  4

Epoch 15: saving model to model_init_2024-12-2619_34_37.119648/model-00015-2.55539-0.45249-2.43803-0.51000.keras
27/27 ━━━━━━━━━━━━━━━━━━━━ 148s 5s/step - categorical_accuracy: 0.4576 - loss: 2.5383 - val_categorical_accuracy: 0.5100 - val_loss: 2.4380 - learning_rate: 1.0000e-04


So we have got the training accuarcy of 45 % and validation accuracy of 51 % in the base model

#### Experiment - 3
Conv 3D Model with 20 epochs, 30 batch size

Without dropouts in Conv layer and with batch normalization

Input image size 120x120 , adam optimiser with learning rate 0.0001, 18 images as input out of 30

In [98]:
#train_doc = np.random.permutation(open('Project_data_Final/train_100.csv').readlines())
#val_doc = np.random.permutation(open('Project_data_Final/val_50.csv').readlines())

train_doc = np.random.permutation(open('Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('Project_data/val.csv').readlines())

batch_size = 30
num_epochs = 20
print ('# epochs =', num_epochs)

# epochs = 20


In [97]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv3D, MaxPooling3D, BatchNormalization, Activation, Dropout, GlobalAveragePooling3D, Dense
from tensorflow.keras import regularizers

# Hyperparameters
l2_reg = 0.01
num_classes = 5

# Initialize the model
model = Sequential()

# First Conv3D layer
model.add(Conv3D(
    16, 
    (3, 3, 3), 
    padding='same', 
    input_shape=(18, 120, 120, 3), 
    kernel_regularizer=regularizers.l2(l2_reg)
))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

# Second Conv3D layer
model.add(Conv3D(
    32, 
    (3, 3, 3), 
    padding='same', 
    kernel_regularizer=regularizers.l2(l2_reg)
))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

# Third Conv3D layer
model.add(Conv3D(
    64, 
    (3, 3, 3), 
    padding='same', 
    kernel_regularizer=regularizers.l2(l2_reg)
))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

# Global Average Pooling and Fully Connected Layers
model.add(GlobalAveragePooling3D())
model.add(Dense(128, activation='relu', kernel_regularizer=regularizers.l2(l2_reg)))

# Output Layer
model.add(Dense(num_classes, activation='softmax'))


In [98]:
# Compile the model with an optimizer
optimiser = optimizers.Adam(learning_rate=0.0001)  # You can experiment with the learning rate
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

# Print the model summary to check the total number of parameters
print(model.summary())

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv3d_18 (Conv3D)                   │ (None, 18, 120, 120, 16)    │           1,312 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_18               │ (None, 18, 120, 120, 16)    │              64 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_18 (Activation)           │ (None, 18, 120, 120, 16)    │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling3d_18 (MaxPooling3D)      │ (None, 9, 60, 60, 16)       │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv3d_19 (Conv3D)                   │ (None, 9, 60, 60, 32)       │          13,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_19               │ (None, 9, 60, 60, 32)       │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_19 (Activation)           │ (None, 9, 60, 60, 32)       │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling3d_19 (MaxPooling3D)      │ (None, 4, 30, 30, 32)       │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv3d_20 (Conv3D)                   │ (None, 4, 30, 30, 64)       │          55,360 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_20               │ (None, 4, 30, 30, 64)       │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_20 (Activation)           │ (None, 4, 30, 30, 64)       │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling3d_20 (MaxPooling3D)      │ (None, 2, 15, 15, 64)       │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling3d_6           │ (None, 64)                  │               0 │
│ (GlobalAveragePooling3D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_12 (Dense)                     │ (None, 128)                 │           8,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_13 (Dense)                     │ (None, 5)                   │             645 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 79,941 (312.27 KB)

 Trainable params: 79,717 (311.39 KB)

 Non-trainable params: 224 (896.00 B)

None


In [99]:
train_generator = generator_2(train_path, train_doc, batch_size)
val_generator = generator_2(val_path, val_doc, batch_size)

In [100]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

print('Steps per epoch:', steps_per_epoch)
print('Validation steps:', validation_steps)

Steps per epoch: 23
Validation steps: 4


In [101]:
model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, initial_epoch=0)

Source path =  Project_data_Final/train ; batch size = 30


C:\Users\upujaari\AppData\Local\Temp\ipykernel_31880\669548316.py:22: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(img_path).astype(np.float32)  # Read the image using imageio


Epoch 1/20
21/23 ━━━━━━━━━━━━━━━━━━━━ 7s 4s/step - categorical_accuracy: 0.1950 - loss: 3.3500 

C:\Users\upujaari\AppData\Local\Temp\ipykernel_31880\669548316.py:59: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(img_path).astype(np.float32)  # Read the image


num_batches =  22
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - categorical_accuracy: 0.1969 - loss: 3.3429Source path =  Project_data_Final/val ; batch size = 30
num_batches =  3

Epoch 1: saving model to model_init_2024-12-2619_21_51.635635/model-00001-3.26843-0.21719-3.13768-0.18000.keras
23/23 ━━━━━━━━━━━━━━━━━━━━ 99s 4s/step - categorical_accuracy: 0.1977 - loss: 3.3398 - val_categorical_accuracy: 0.1800 - val_loss: 3.1377 - learning_rate: 1.0000e-04
Epoch 2/20
21/23 ━━━━━━━━━━━━━━━━━━━━ 7s 4s/step - categorical_accuracy: 0.3225 - loss: 3.0582 num_batches =  22
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - categorical_accuracy: 0.3239 - loss: 3.0559num_batches =  3

Epoch 2: saving model to model_init_2024-12-2619_21_51.635635/model-00002-3.03176-0.33786-3.08669-0.28000.keras
23/23 ━━━━━━━━━━━━━━━━━━━━ 91s 4s/step - categorical_accuracy: 0.3244 - loss: 3.0549 - val_categorical_accuracy: 0.2800 - val_loss: 3.0867 - learning_rate: 1.0000e-04
Epoch 3/20
21/23 ━━━━━━━━━━━━━━━━━━━━ 7s 4s/step - ca

So we have got the training accuarcy of 89 % and validation accuracy of 76 % in the base model

#### Experiment - 4
Conv 3D Model with 50 epochs, 30 batch size

Without dropouts in Conv layer and with batch normalization

Input image size 160x160 , adam optimiser with learning rate 0.0001, 18 images as input out of 30

In [95]:
#train_doc = np.random.permutation(open('Project_data_Final/train_100.csv').readlines())
#val_doc = np.random.permutation(open('Project_data_Final/val_50.csv').readlines())

train_doc = np.random.permutation(open('Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('Project_data/val.csv').readlines())

batch_size = 30
num_epochs = 50
print ('# epochs =', num_epochs)

# epochs = 50


In [16]:
def generator_3(source_path, folder_list, batch_size):
    print('Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [0,1,2,4,5,6,8,9,10,12,14,16,18,20,22,24,26,27,28,29]
    #list(range(30))  # We will use 30 frames per video (adjustable)
    x, y, z = len(img_idx), 160, 160  # x is the number of images, y and z are the target dimensions

    while True:
        t = np.random.permutation(folder_list)  # Shuffle the list of folders (videos)
        num_batches = len(folder_list) // batch_size  # Calculate number of full batches
        

        for batch in range(num_batches):  # Iterate over the number of batches
            batch_data = np.zeros((batch_size, x, y, z, 3))  # Initialize batch data (40 samples, 30 frames, 160x160, 3 channels)
            batch_labels = np.zeros((batch_size, 5))  # Initialize batch labels (one-hot encoding for 5 classes)
            
            for folder in range(batch_size):  # Iterate over the batch size (40 samples)
                folder_path = source_path + '/' + t[folder + (batch * batch_size)].split(';')[0]
                imgs = os.listdir(folder_path)  # Read all image files in the folder

                for idx, item in enumerate(img_idx):  # Iterate over the frames/images (using img_idx)
                    img_path = os.path.join(folder_path, imgs[item])  # Get the image path
                    image = imageio.imread(img_path).astype(np.float32)  # Read the image using imageio
                    
                    # Crop and resize images to ensure uniformity in shape
                    image = crop_img(image, 0.10)  # Crop 10% from both sides
                    
                    # Resize the image to 160x160 using PIL 
                    image = Image.fromarray(image.astype(np.uint8))  # Convert numpy array to PIL image
                    image = image.resize((y, z), Image.Resampling.LANCZOS)  # Resize using Pillow
                    image = np.array(image).astype(np.float32)  # Convert back to numpy array
                    
                    # Normalize the image by mean normalization (subtract mean and divide by standard deviation)
                    image = (image - np.mean(image)) / np.std(image)  # Mean normalization

                    # Load the image into the 3 RGB channels
                    # Ensure we only store each colour channel separately
                    batch_data[folder, idx, :, :, 0] = image[:, :, 0]  # Red channel
                    batch_data[folder, idx, :, :, 1] = image[:, :, 1]  # Green channel
                    batch_data[folder, idx, :, :, 2] = image[:, :, 2]  # Blue channel

                # Set the one-hot encoding for the label (class)
                class_idx = int(t[folder + (batch * batch_size)].strip().split(';')[2])  # Get the class index from the CSV file
                batch_labels[folder, class_idx] = 1  # One-hot encoding for the class label

            yield batch_data, batch_labels  # Yield the batch of data and labels

        # Handle remaining data points (if not a perfect multiple of batch_size)
        if len(folder_list) % batch_size != 0:
            remaining_samples = len(folder_list) % batch_size
            batch_data_remaining = np.zeros((remaining_samples, x, y, z, 3))  # Remaining batch data
            batch_labels_remaining = np.zeros((remaining_samples, 5))  # Remaining batch labels

            for folder in range(remaining_samples):  # Process the remaining samples
                folder_path = source_path + '/' + t[folder + (num_batches * batch_size)].split(';')[0]
                imgs = os.listdir(folder_path)  # Read all image files in the folder

                for idx, item in enumerate(img_idx):  # Iterate over frames/images of a folder
                    img_path = os.path.join(folder_path, imgs[item])  # Get the image path
                    image = imageio.imread(img_path).astype(np.float32)  # Read the image

                    # Crop and resize the images to ensure uniform shape
                    image = crop_img(image, 0.10)  # Crop 10% from both sides
                    
                    # Resize the image to 160x160 using PIL
                    image = Image.fromarray(image.astype(np.uint8))  # Convert numpy array to PIL image
                    image = image.resize((y, z), Image.Resampling.LANCZOS)  # Resize using Pillow
                    image = np.array(image).astype(np.float32)  # Convert back to numpy array
                    
                    # Normalize the image
                    image = (image - np.mean(image)) / np.std(image)  # Mean normalization

                    # Load the image into the 3 RGB channels
                    # Ensure we only store each colour channel separately
                    batch_data[folder, idx, :, :, 0] = image[:, :, 0]  # Red channel
                    batch_data[folder, idx, :, :, 1] = image[:, :, 1]  # Green channel
                    batch_data[folder, idx, :, :, 2] = image[:, :, 2]  # Blue channel

                # Set the one-hot encoding for the class
                class_idx = int(t[folder + (num_batches * batch_size)].strip().split(';')[2])
                batch_labels_remaining[folder, class_idx] = 1  # One-hot encoding for the class label
                
            yield batch_data_remaining, batch_labels_remaining  # Yield the final batch with the remaining data


In [105]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv3D, MaxPooling3D, BatchNormalization, Activation, Dropout, GlobalAveragePooling3D, Dense
from tensorflow.keras import regularizers

# Hyperparameters
l2_reg = 0.01
num_classes = 5

# Initialize the model
model = Sequential()

# First Conv3D layer
model.add(Conv3D(
    16, 
    (3, 3, 3), 
    padding='same', 
    input_shape=(18, 160, 160, 3), 
    kernel_regularizer=regularizers.l2(l2_reg)
))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

# Second Conv3D layer
model.add(Conv3D(
    32, 
    (3, 3, 3), 
    padding='same', 
    kernel_regularizer=regularizers.l2(l2_reg)
))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

# Third Conv3D layer
model.add(Conv3D(
    64, 
    (3, 3, 3), 
    padding='same', 
    kernel_regularizer=regularizers.l2(l2_reg)
))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

# Global Average Pooling and Fully Connected Layers
model.add(GlobalAveragePooling3D())
model.add(Dense(128, activation='relu', kernel_regularizer=regularizers.l2(l2_reg)))

# Output Layer
model.add(Dense(num_classes, activation='softmax'))


In [106]:
# Compile the model with an optimizer
optimiser = optimizers.Adam(learning_rate=0.0001)  # You can experiment with the learning rate
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

# Print the model summary to check the total number of parameters
print(model.summary())

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv3d_21 (Conv3D)                   │ (None, 18, 160, 160, 16)    │           1,312 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_21               │ (None, 18, 160, 160, 16)    │              64 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_21 (Activation)           │ (None, 18, 160, 160, 16)    │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling3d_21 (MaxPooling3D)      │ (None, 9, 80, 80, 16)       │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv3d_22 (Conv3D)                   │ (None, 9, 80, 80, 32)       │          13,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_22               │ (None, 9, 80, 80, 32)       │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_22 (Activation)           │ (None, 9, 80, 80, 32)       │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling3d_22 (MaxPooling3D)      │ (None, 4, 40, 40, 32)       │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv3d_23 (Conv3D)                   │ (None, 4, 40, 40, 64)       │          55,360 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_23               │ (None, 4, 40, 40, 64)       │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_23 (Activation)           │ (None, 4, 40, 40, 64)       │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling3d_23 (MaxPooling3D)      │ (None, 2, 20, 20, 64)       │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling3d_7           │ (None, 64)                  │               0 │
│ (GlobalAveragePooling3D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_14 (Dense)                     │ (None, 128)                 │           8,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_15 (Dense)                     │ (None, 5)                   │             645 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 79,941 (312.27 KB)

 Trainable params: 79,717 (311.39 KB)

 Non-trainable params: 224 (896.00 B)

None


In [107]:
train_generator = generator_3(train_path, train_doc, batch_size)
val_generator = generator_3(val_path, val_doc, batch_size)

In [108]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

print('Steps per epoch:', steps_per_epoch)
print('Validation steps:', validation_steps)

Steps per epoch: 23
Validation steps: 4


In [109]:
model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, initial_epoch=0)

Source path =  Project_data_Final/train ; batch size = 30


C:\Users\upujaari\AppData\Local\Temp\ipykernel_31880\2363037342.py:22: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(img_path).astype(np.float32)  # Read the image using imageio


Epoch 1/50
21/23 ━━━━━━━━━━━━━━━━━━━━ 12s 6s/step - categorical_accuracy: 0.1887 - loss: 3.4841

C:\Users\upujaari\AppData\Local\Temp\ipykernel_31880\2363037342.py:59: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(img_path).astype(np.float32)  # Read the image


23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - categorical_accuracy: 0.1910 - loss: 3.4673Source path =  Project_data_Final/val ; batch size = 30

Epoch 1: saving model to model_init_2024-12-2619_21_51.635635/model-00001-3.29081-0.21418-3.13427-0.21000.keras
23/23 ━━━━━━━━━━━━━━━━━━━━ 155s 7s/step - categorical_accuracy: 0.1919 - loss: 3.4600 - val_categorical_accuracy: 0.2100 - val_loss: 3.1343 - learning_rate: 1.0000e-04
Epoch 2/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - categorical_accuracy: 0.2971 - loss: 3.0345
Epoch 2: saving model to model_init_2024-12-2619_21_51.635635/model-00002-3.02796-0.31523-3.08710-0.20000.keras
23/23 ━━━━━━━━━━━━━━━━━━━━ 139s 6s/step - categorical_accuracy: 0.2978 - loss: 3.0343 - val_categorical_accuracy: 0.2000 - val_loss: 3.0871 - learning_rate: 1.0000e-04
Epoch 3/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - categorical_accuracy: 0.4299 - loss: 2.9267
Epoch 3: saving model to model_init_2024-12-2619_21_51.635635/model-00003-2.90942-0.44193-3.03834-0.20000.kera

So we have got the training accuarcy of 93 % and validation accuracy of 77 % in the base model

#### Experiment 5
CNN + LSTM Model with 30 epochs, 30 batch size

Without dropouts in Conv layer and with batch normalization

Input image size 120x120 , adam optimiser with learning rate 0.0001, 20 images as input out of 30

In [14]:
train_doc = np.random.permutation(open('Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('Project_data/val.csv').readlines())
batch_size = 30
num_epochs = 30
print ('# epochs =', num_epochs)

# epochs = 30


In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Activation, Flatten, Dense, Dropout, LSTM, TimeDistributed
from tensorflow.keras import Input, regularizers

# Hyperparameters
# dropout_rate = 0.5
l2_reg = 0.01
num_classes = 5

# Input shape: (number of frames, height, width, channels)
input_shape = (20, 120, 120, 3)

# Initialize the model
model = Sequential()

# TimeDistributed CNN Layers
model.add(TimeDistributed(Conv2D(32, (3, 3), padding='same', kernel_regularizer=regularizers.l2(l2_reg)), input_shape=input_shape))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(Activation('relu')))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))

model.add(TimeDistributed(Conv2D(64, (3, 3), padding='same', kernel_regularizer=regularizers.l2(l2_reg))))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(Activation('relu')))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))

model.add(TimeDistributed(Conv2D(128, (3, 3), padding='same', kernel_regularizer=regularizers.l2(l2_reg))))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(Activation('relu')))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))

# Flatten spatial features
model.add(TimeDistributed(Flatten()))

# LSTM Layers
model.add(LSTM(128, return_sequences=False, kernel_regularizer=regularizers.l2(l2_reg)))
# model.add(Dropout(dropout_rate))

# Fully Connected Layer
model.add(Dense(128, activation='relu', kernel_regularizer=regularizers.l2(l2_reg)))
# model.add(Dropout(dropout_rate))

# Output Layer
model.add(Dense(num_classes, activation='softmax'))


/Users/rajashekar/Downloads/Softwares/Anaconda/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/wrapper.py:27: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [16]:
# Compile the model with an optimizer
optimiser = optimizers.Adam(learning_rate=0.0001)  # You can experiment with the learning rate
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

# Print the model summary to check the total number of parameters
print(model.summary())

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ time_distributed                │ (None, 20, 120, 120,   │           896 │
│ (TimeDistributed)               │ 32)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_1              │ (None, 20, 120, 120,   │           128 │
│ (TimeDistributed)               │ 32)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_2              │ (None, 20, 120, 120,   │             0 │
│ (TimeDistributed)               │ 32)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_3              │ (None, 20, 60, 60, 32) │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_4              │ (None, 20, 60, 60, 64) │        18,496 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_5              │ (None, 20, 60, 60, 64) │           256 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_6              │ (None, 20, 60, 60, 64) │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_7              │ (None, 20, 30, 30, 64) │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_8              │ (None, 20, 30, 30,     │        73,856 │
│ (TimeDistributed)               │ 128)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_9              │ (None, 20, 30, 30,     │           512 │
│ (TimeDistributed)               │ 128)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_10             │ (None, 20, 30, 30,     │             0 │
│ (TimeDistributed)               │ 128)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_11             │ (None, 20, 15, 15,     │             0 │
│ (TimeDistributed)               │ 128)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_12             │ (None, 20, 28800)      │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 128)            │    14,811,648 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 5)              │           645 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,922,949 (56.93 MB)

 Trainable params: 14,922,501 (56.92 MB)

 Non-trainable params: 448 (1.75 KB)

None


In [18]:
train_generator = generator_2(train_path, train_doc, batch_size)
val_generator = generator_2(val_path, val_doc, batch_size)

In [19]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

print('Steps per epoch:', steps_per_epoch)
print('Validation steps:', validation_steps)

Steps per epoch: 23
Validation steps: 4


In [20]:
model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, initial_epoch=0)

Source path =  Project_data/train ; batch size = 30
num_batches =  22


/var/folders/th/mw8md0ks7rsdgy45pvbb9cgw0000gn/T/ipykernel_1039/4273982213.py:22: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(img_path).astype(np.float32)  # Read the image using imageio


Epoch 1/30
21/23 ━━━━━━━━━━━━━━━━━━━━ 54s 27s/step - categorical_accuracy: 0.2398 - loss: 13.9791 

/var/folders/th/mw8md0ks7rsdgy45pvbb9cgw0000gn/T/ipykernel_1039/4273982213.py:59: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(img_path).astype(np.float32)  # Read the image


22/23 ━━━━━━━━━━━━━━━━━━━━ 27s 27s/step - categorical_accuracy: 0.2424 - loss: 13.9596num_batches =  22
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 26s/step - categorical_accuracy: 0.2447 - loss: 13.9416 Source path =  Project_data/val ; batch size = 30
num_batches =  3
num_batches =  3

Epoch 1: saving model to model_init_2024-12-2707_52_10.735881/model-00001-13.54548-0.29563-12.55466-0.35000.keras
23/23 ━━━━━━━━━━━━━━━━━━━━ 617s 27s/step - categorical_accuracy: 0.2468 - loss: 13.9251 - val_categorical_accuracy: 0.3500 - val_loss: 12.5547 - learning_rate: 1.0000e-04
Epoch 2/30
22/23 ━━━━━━━━━━━━━━━━━━━━ 27s 28s/step - categorical_accuracy: 0.5322 - loss: 11.9925num_batches =  22
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 26s/step - categorical_accuracy: 0.5327 - loss: 11.9732 num_batches =  3

Epoch 2: saving model to model_init_2024-12-2707_52_10.735881/model-00002-11.54871-0.54449-10.75422-0.48000.keras
23/23 ━━━━━━━━━━━━━━━━━━━━ 617s 27s/step - categorical_accuracy: 0.5332 - loss: 11.9556 - val_categorical_a

23/23 ━━━━━━━━━━━━━━━━━━━━ 630s 27s/step - categorical_accuracy: 0.9997 - loss: 3.3397 - val_categorical_accuracy: 0.5300 - val_loss: 4.3528 - learning_rate: 1.0000e-04
Epoch 15/30
22/23 ━━━━━━━━━━━━━━━━━━━━ 28s 28s/step - categorical_accuracy: 1.0000 - loss: 3.1696num_batches =  22
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 27s/step - categorical_accuracy: 0.9999 - loss: 3.1682 num_batches =  3

Epoch 15: saving model to model_init_2024-12-2707_52_10.735881/model-00015-3.13556-0.99698-4.00208-0.55000.keras
23/23 ━━━━━━━━━━━━━━━━━━━━ 631s 27s/step - categorical_accuracy: 0.9997 - loss: 3.1668 - val_categorical_accuracy: 0.5500 - val_loss: 4.0021 - learning_rate: 1.0000e-04
Epoch 16/30
22/23 ━━━━━━━━━━━━━━━━━━━━ 28s 28s/step - categorical_accuracy: 1.0000 - loss: 3.0030num_batches =  22
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 27s/step - categorical_accuracy: 0.9999 - loss: 3.0017 num_batches =  3

Epoch 16: saving model to model_init_2024-12-2707_52_10.735881/model-00016-2.97235-0.99849-3.99457-0.52000.keras


So we have got the training accuarcy of 99 % and validation accuracy of 60 % in the base model

#### Experiment 6
CNN + LSTM Model with 50 epochs, 30 batch size

Without dropouts in Conv layer and with batch normalization

Input image size 160x160 , adam optimiser with learning rate 0.0001, 20 images as input out of 30

In [23]:
train_doc_100 = np.random.permutation(open('Project_data/train_100.csv').readlines())
val_doc_50 = np.random.permutation(open('Project_data/val_50.csv').readlines())
batch_size = 30
num_epochs = 50
print ('# epochs =', num_epochs)

# epochs = 50


In [23]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Activation, Flatten, Dense, Dropout, LSTM, TimeDistributed
from tensorflow.keras import Input, regularizers

# Hyperparameters
# dropout_rate = 0.5
l2_reg = 0.01
num_classes = 5

# Input shape: (number of frames, height, width, channels)
input_shape = (20, 160, 160, 3)

# Initialize the model
model = Sequential()

# TimeDistributed CNN Layers
model.add(TimeDistributed(Conv2D(32, (3, 3), padding='same', kernel_regularizer=regularizers.l2(l2_reg)), input_shape=input_shape))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(Activation('relu')))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))

model.add(TimeDistributed(Conv2D(64, (3, 3), padding='same', kernel_regularizer=regularizers.l2(l2_reg))))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(Activation('relu')))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))

model.add(TimeDistributed(Conv2D(128, (3, 3), padding='same', kernel_regularizer=regularizers.l2(l2_reg))))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(Activation('relu')))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))

# Flatten spatial features
model.add(TimeDistributed(Flatten()))

# LSTM Layers
model.add(LSTM(128, return_sequences=False, kernel_regularizer=regularizers.l2(l2_reg)))
# model.add(Dropout(dropout_rate))

# Fully Connected Layer
model.add(Dense(128, activation='relu', kernel_regularizer=regularizers.l2(l2_reg)))
# model.add(Dropout(dropout_rate))

# Output Layer
model.add(Dense(num_classes, activation='softmax'))


In [25]:
# Compile the model with an optimizer
optimiser = optimizers.Adam(learning_rate=0.0001)  # You can experiment with the learning rate
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

# Print the model summary to check the total number of parameters
print(model.summary())

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ time_distributed                │ (None, 20, 160, 160,   │           896 │
│ (TimeDistributed)               │ 32)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_1              │ (None, 20, 160, 160,   │           128 │
│ (TimeDistributed)               │ 32)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_2              │ (None, 20, 160, 160,   │             0 │
│ (TimeDistributed)               │ 32)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_3              │ (None, 20, 80, 80, 32) │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_4              │ (None, 20, 80, 80, 64) │        18,496 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_5              │ (None, 20, 80, 80, 64) │           256 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_6              │ (None, 20, 80, 80, 64) │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_7              │ (None, 20, 40, 40, 64) │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_8              │ (None, 20, 40, 40,     │        73,856 │
│ (TimeDistributed)               │ 128)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_9              │ (None, 20, 40, 40,     │           512 │
│ (TimeDistributed)               │ 128)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_10             │ (None, 20, 40, 40,     │             0 │
│ (TimeDistributed)               │ 128)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_11             │ (None, 20, 20, 20,     │             0 │
│ (TimeDistributed)               │ 128)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_12             │ (None, 20, 51200)      │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 128)            │    26,280,448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 5)              │           645 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 26,391,749 (100.68 MB)

 Trainable params: 26,391,301 (100.67 MB)

 Non-trainable params: 448 (1.75 KB)

None


In [26]:
train_generator = generator_3(train_path, train_doc_100, batch_size)
val_generator = generator_3(val_path, val_doc_50, batch_size)

In [27]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

print('Steps per epoch:', steps_per_epoch)
print('Validation steps:', validation_steps)

Steps per epoch: 23
Validation steps: 4


In [28]:
model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, initial_epoch=0)

Source path =  Project_data/train ; batch size = 30


/var/folders/th/mw8md0ks7rsdgy45pvbb9cgw0000gn/T/ipykernel_1081/2363037342.py:22: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(img_path).astype(np.float32)  # Read the image using imageio


Epoch 1/50
21/23 ━━━━━━━━━━━━━━━━━━━━ 2:22 71s/step - categorical_accuracy: 0.2824 - loss: 14.0048

/var/folders/th/mw8md0ks7rsdgy45pvbb9cgw0000gn/T/ipykernel_1081/2363037342.py:59: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(img_path).astype(np.float32)  # Read the image


23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 68s/step - categorical_accuracy: 0.2858 - loss: 13.9575  Source path =  Project_data/val ; batch size = 30

Epoch 1: saving model to model_init_2024-12-2714_36_52.054119/model-00001-13.45822-0.32127-12.18563-0.37000.keras
23/23 ━━━━━━━━━━━━━━━━━━━━ 1596s 69s/step - categorical_accuracy: 0.2873 - loss: 13.9367 - val_categorical_accuracy: 0.3700 - val_loss: 12.1856 - learning_rate: 1.0000e-04
Epoch 2/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 69s/step - categorical_accuracy: 0.5515 - loss: 11.5095  
Epoch 2: saving model to model_init_2024-12-2714_36_52.054119/model-00002-11.01347-0.52640-9.95407-0.50000.keras
23/23 ━━━━━━━━━━━━━━━━━━━━ 1606s 70s/step - categorical_accuracy: 0.5505 - loss: 11.4888 - val_categorical_accuracy: 0.5000 - val_loss: 9.9541 - learning_rate: 1.0000e-04
Epoch 3/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 69s/step - categorical_accuracy: 0.6552 - loss: 9.3750  
Epoch 3: saving model to model_init_2024-12-2714_36_52.054119/model-00003-8.98669-0.66516-8.330

23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 69s/step - categorical_accuracy: 0.9812 - loss: 1.2706  
Epoch 37: saving model to model_init_2024-12-2714_36_52.054119/model-00037-1.28479-0.97285-2.56614-0.50000.keras
23/23 ━━━━━━━━━━━━━━━━━━━━ 1610s 70s/step - categorical_accuracy: 0.9808 - loss: 1.2712 - val_categorical_accuracy: 0.5000 - val_loss: 2.5661 - learning_rate: 1.0000e-04
Epoch 38/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 69s/step - categorical_accuracy: 0.9914 - loss: 1.2326  
Epoch 38: saving model to model_init_2024-12-2714_36_52.054119/model-00038-1.22061-0.99095-2.11267-0.58000.keras
23/23 ━━━━━━━━━━━━━━━━━━━━ 1612s 70s/step - categorical_accuracy: 0.9914 - loss: 1.2321 - val_categorical_accuracy: 0.5800 - val_loss: 2.1127 - learning_rate: 1.0000e-04
Epoch 39/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 69s/step - categorical_accuracy: 0.9998 - loss: 1.1382  
Epoch 39: saving model to model_init_2024-12-2714_36_52.054119/model-00039-1.12803-0.99548-2.10999-0.66000.keras
23/23 ━━━━━━━━━━━━━━━━━━━━ 1614s 70s

So we have got the training accuarcy of 99 % and validation accuracy of 67 % in the base model

#### Experiment 7
CNN + LSTM Model with 20 epochs, 20 batch size

Without dropouts in Conv layer and with batch normalization

Input image size 160x160 , adam optimiser with learning rate 0.0001, 20 images as input out of 30

In [96]:
train_doc = np.random.permutation(open('Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('Project_data/val.csv').readlines())
batch_size = 20
num_epochs = 20
print ('# epochs =', num_epochs)

# epochs = 20


In [125]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Conv2D, MaxPooling2D, Flatten, BatchNormalization, Activation, TimeDistributed
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras import optimizers

l2_reg = 0.01
num_classes = 5

# Input shape: (number of frames, height, width, channels)
input_shape = (20, 160, 160, 3)

GRU_model = Sequential()

GRU_model.add(TimeDistributed(Conv2D(16, (3, 3) , padding='same', activation='relu'),
                          input_shape=(input_shape)))
GRU_model.add(TimeDistributed(BatchNormalization()))
GRU_model.add(TimeDistributed(MaxPooling2D((2, 2))))

GRU_model.add(TimeDistributed(Conv2D(32, (3, 3) , padding='same', activation='relu')))
GRU_model.add(TimeDistributed(BatchNormalization()))
GRU_model.add(TimeDistributed(MaxPooling2D((2, 2))))

GRU_model.add(TimeDistributed(Conv2D(64, (3, 3) , padding='same', activation='relu')))
GRU_model.add(TimeDistributed(BatchNormalization()))
GRU_model.add(TimeDistributed(MaxPooling2D((2, 2))))

GRU_model.add(TimeDistributed(Conv2D(128, (3, 3) , padding='same', activation='relu')))
GRU_model.add(TimeDistributed(BatchNormalization()))
GRU_model.add(TimeDistributed(MaxPooling2D((2, 2))))


GRU_model.add(TimeDistributed(Flatten()))


GRU_model.add(GRU(128))

GRU_model.add(Dense(128,activation='relu'))

GRU_model.add(Dense(5, activation='softmax'))

C:\Users\upujaari\AppData\Local\anaconda3\Lib\site-packages\keras\src\layers\core\wrapper.py:27: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [126]:
# Compile the model with an optimizer
optimiser = optimizers.Adam(learning_rate=0.0001)  # You can experiment with the learning rate
GRU_model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

# Print the model summary to check the total number of parameters
print(GRU_model.summary())

Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ time_distributed (TimeDistributed)   │ (None, 20, 160, 160, 16)    │             448 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_1 (TimeDistributed) │ (None, 20, 160, 160, 16)    │              64 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_2 (TimeDistributed) │ (None, 20, 80, 80, 16)      │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_3 (TimeDistributed) │ (None, 20, 80, 80, 32)      │           4,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_4 (TimeDistributed) │ (None, 20, 80, 80, 32)      │             128 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_5 (TimeDistributed) │ (None, 20, 40, 40, 32)      │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_6 (TimeDistributed) │ (None, 20, 40, 40, 64)      │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_7 (TimeDistributed) │ (None, 20, 40, 40, 64)      │             256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_8 (TimeDistributed) │ (None, 20, 20, 20, 64)      │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_9 (TimeDistributed) │ (None, 20, 20, 20, 128)     │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_10                  │ (None, 20, 20, 20, 128)     │             512 │
│ (TimeDistributed)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_11                  │ (None, 20, 10, 10, 128)     │               0 │
│ (TimeDistributed)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_12                  │ (None, 20, 12800)           │               0 │
│ (TimeDistributed)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru (GRU)                            │ (None, 128)                 │       4,965,120 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_16 (Dense)                     │ (None, 128)                 │          16,512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_17 (Dense)                     │ (None, 5)                   │             645 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 5,080,677 (19.38 MB)

 Trainable params: 5,080,197 (19.38 MB)

 Non-trainable params: 480 (1.88 KB)

None


In [127]:
train_generator = generator_3(train_path, train_doc, batch_size)
val_generator = generator_3(val_path, val_doc, batch_size)

In [128]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

print('Steps per epoch:', steps_per_epoch)
print('Validation steps:', validation_steps)

Steps per epoch: 34
Validation steps: 5


In [129]:
GRU_model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, initial_epoch=0)

Source path =  Project_data_Final/train ; batch size = 20


C:\Users\upujaari\AppData\Local\Temp\ipykernel_31880\2363037342.py:22: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(img_path).astype(np.float32)  # Read the image using imageio


Epoch 1/20
32/34 ━━━━━━━━━━━━━━━━━━━━ 23s 12s/step - categorical_accuracy: 0.3449 - loss: 1.5167

C:\Users\upujaari\AppData\Local\Temp\ipykernel_31880\2363037342.py:59: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(img_path).astype(np.float32)  # Read the image


34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 11s/step - categorical_accuracy: 0.3513 - loss: 1.5067 Source path =  Project_data_Final/val ; batch size = 20

Epoch 1: saving model to model_init_2024-12-2816_47_18.992975/model-00001-1.34730-0.45400-1.19947-0.52000.keras
34/34 ━━━━━━━━━━━━━━━━━━━━ 467s 12s/step - categorical_accuracy: 0.3542 - loss: 1.5022 - val_categorical_accuracy: 0.5200 - val_loss: 1.1995 - learning_rate: 1.0000e-04
Epoch 2/20
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 11s/step - categorical_accuracy: 0.8271 - loss: 0.6712 
Epoch 2: saving model to model_init_2024-12-2816_47_18.992975/model-00002-0.61812-0.84465-1.16294-0.56000.keras
34/34 ━━━━━━━━━━━━━━━━━━━━ 382s 11s/step - categorical_accuracy: 0.8276 - loss: 0.6697 - val_categorical_accuracy: 0.5600 - val_loss: 1.1629 - learning_rate: 1.0000e-04
Epoch 3/20
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 11s/step - categorical_accuracy: 0.9555 - loss: 0.3274 
Epoch 3: saving model to model_init_2024-12-2816_47_18.992975/model-00003-0.29871-0.96531-0.93549-0.70

So we have got the training accuarcy of 99 % and validation accuracy of 65 % in the base model

#### Experiment 8 - Adding dropout layer, increasing number of layers
CNN + GRU Model with 20 epochs, 30 batch size

Without dropouts in Conv layer and with batch normalization

Input image size 160x160 , adam optimiser with learning rate 0.0001, 20 images as input out of 30

In [97]:
train_doc = np.random.permutation(open('Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('Project_data/val.csv').readlines())
batch_size = 30
num_epochs = 20
print ('# epochs =', num_epochs)

# epochs = 20


In [20]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Conv2D, MaxPooling2D, Flatten, BatchNormalization, Activation, TimeDistributed
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras import optimizers

l2_reg = 0.01
num_classes = 5

# Input shape: (number of frames, height, width, channels)
input_shape = (20, 160, 160, 3)

GRU_model = Sequential()

GRU_model.add(TimeDistributed(Conv2D(16, (3, 3) , padding='same', activation='relu'),
                          input_shape=(input_shape)))
GRU_model.add(TimeDistributed(BatchNormalization()))
GRU_model.add(TimeDistributed(MaxPooling2D((2, 2))))

GRU_model.add(TimeDistributed(Conv2D(32, (3, 3) , padding='same', activation='relu')))
GRU_model.add(TimeDistributed(BatchNormalization()))
GRU_model.add(TimeDistributed(MaxPooling2D((2, 2))))

GRU_model.add(TimeDistributed(Conv2D(32, (3, 3) , padding='same', activation='relu')))
GRU_model.add(TimeDistributed(BatchNormalization()))
GRU_model.add(TimeDistributed(MaxPooling2D((2, 2))))

GRU_model.add(TimeDistributed(Conv2D(64, (3, 3) , padding='same', activation='relu')))
GRU_model.add(TimeDistributed(BatchNormalization()))
GRU_model.add(TimeDistributed(MaxPooling2D((2, 2))))

GRU_model.add(TimeDistributed(Conv2D(64, (3, 3) , padding='same', activation='relu')))
GRU_model.add(TimeDistributed(BatchNormalization()))
GRU_model.add(TimeDistributed(MaxPooling2D((2, 2))))

GRU_model.add(TimeDistributed(Conv2D(128, (3, 3) , padding='same', activation='relu')))
GRU_model.add(TimeDistributed(BatchNormalization()))
GRU_model.add(TimeDistributed(MaxPooling2D((2, 2))))

GRU_model.add(TimeDistributed(Conv2D(128, (3, 3) , padding='same', activation='relu')))
GRU_model.add(TimeDistributed(BatchNormalization()))
GRU_model.add(TimeDistributed(MaxPooling2D((2, 2))))


GRU_model.add(TimeDistributed(Flatten()))


GRU_model.add(GRU(128))

GRU_model.add(Dense(128,activation='relu'))
GRU_model.add(Dropout(0.25))

GRU_model.add(Dense(5, activation='softmax'))

/Users/rajashekar/Downloads/Softwares/Anaconda/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/wrapper.py:27: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


NameError: name 'Dropout' is not defined

In [147]:
# Compile the model with an optimizer
optimiser = optimizers.Adam(learning_rate=0.0001)  # You can experiment with the learning rate
GRU_model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

# Print the model summary to check the total number of parameters
print(GRU_model.summary())

Model: "sequential_14"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ time_distributed_61                  │ (None, 20, 160, 160, 16)    │             448 │
│ (TimeDistributed)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_62                  │ (None, 20, 160, 160, 16)    │              64 │
│ (TimeDistributed)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_63                  │ (None, 20, 80, 80, 16)      │               0 │
│ (TimeDistributed)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_64                  │ (None, 20, 80, 80, 32)      │           4,640 │
│ (TimeDistributed)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_65                  │ (None, 20, 80, 80, 32)      │             128 │
│ (TimeDistributed)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_66                  │ (None, 20, 40, 40, 32)      │               0 │
│ (TimeDistributed)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_67                  │ (None, 20, 40, 40, 32)      │           9,248 │
│ (TimeDistributed)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_68                  │ (None, 20, 40, 40, 32)      │             128 │
│ (TimeDistributed)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_69                  │ (None, 20, 20, 20, 32)      │               0 │
│ (TimeDistributed)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_70                  │ (None, 20, 20, 20, 64)      │          18,496 │
│ (TimeDistributed)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_71                  │ (None, 20, 20, 20, 64)      │             256 │
│ (TimeDistributed)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_72                  │ (None, 20, 10, 10, 64)      │               0 │
│ (TimeDistributed)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_73                  │ (None, 20, 10, 10, 64)      │          36,928 │
│ (TimeDistributed)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_74                  │ (None, 20, 10, 10, 64)      │             256 │
│ (TimeDistributed)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

 Total params: 409,285 (1.56 MB)

 Trainable params: 408,357 (1.56 MB)

 Non-trainable params: 928 (3.62 KB)

None


In [148]:
train_generator = generator_3(train_path, train_doc, batch_size)
val_generator = generator_3(val_path, val_doc, batch_size)

In [149]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

print('Steps per epoch:', steps_per_epoch)
print('Validation steps:', validation_steps)

Steps per epoch: 23
Validation steps: 4


In [150]:
GRU_model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, initial_epoch=0)

Source path =  Project_data_Final/train ; batch size = 30


C:\Users\upujaari\AppData\Local\Temp\ipykernel_31880\2363037342.py:22: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(img_path).astype(np.float32)  # Read the image using imageio


Epoch 1/20
21/23 ━━━━━━━━━━━━━━━━━━━━ 1:15 38s/step - categorical_accuracy: 0.2577 - loss: 1.6297

C:\Users\upujaari\AppData\Local\Temp\ipykernel_31880\2363037342.py:59: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(img_path).astype(np.float32)  # Read the image


23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 37s/step - categorical_accuracy: 0.2626 - loss: 1.6241 Source path =  Project_data_Final/val ; batch size = 30

Epoch 1: saving model to model_init_2024-12-2816_47_18.992975/model-00001-1.56560-0.31373-1.41121-0.40000.keras
23/23 ━━━━━━━━━━━━━━━━━━━━ 1005s 38s/step - categorical_accuracy: 0.2647 - loss: 1.6217 - val_categorical_accuracy: 0.4000 - val_loss: 1.4112 - learning_rate: 1.0000e-04
Epoch 2/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 41s/step - categorical_accuracy: 0.5078 - loss: 1.2675 
Epoch 2: saving model to model_init_2024-12-2816_47_18.992975/model-00002-1.23144-0.52338-1.41407-0.35833.keras
23/23 ━━━━━━━━━━━━━━━━━━━━ 966s 41s/step - categorical_accuracy: 0.5084 - loss: 1.2660 - val_categorical_accuracy: 0.3583 - val_loss: 1.4141 - learning_rate: 1.0000e-04
Epoch 3/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 38s/step - categorical_accuracy: 0.6811 - loss: 0.9955 
Epoch 3: saving model to model_init_2024-12-2816_47_18.992975/model-00003-1.00692-0.67270-1.47642-0.3

So we have got the training accuarcy of 99 % and validation accuracy of 48 % in the base model

#### Experiment 9 - Pretrained Mobile net, without learning weights, using LSTM

In [89]:
train_doc = np.random.permutation(open('Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('Project_data/val.csv').readlines())
batch_size = 30
num_epochs = 20
print ('# epochs =', num_epochs)

# epochs = 20


In [90]:
train_generator = generator_3(train_path, train_doc, batch_size)
val_generator = generator_3(val_path, val_doc, batch_size)

In [91]:
# Importing the transfer learning model:
from keras.applications import mobilenet

mobilenet_transfer = mobilenet.MobileNet(weights='imagenet', include_top=False)

/var/folders/th/mw8md0ks7rsdgy45pvbb9cgw0000gn/T/ipykernel_1081/4291270708.py:4: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  mobilenet_transfer = mobilenet.MobileNet(weights='imagenet', include_top=False)


In [92]:
# Building the model :

input_shape = (20,160,160,3)
        
mobileNetModel= Sequential()
mobileNetModel.add(TimeDistributed(mobilenet_transfer,input_shape=(input_shape)))


for layer in mobileNetModel.layers:
    layer.trainable = False


mobileNetModel.add(TimeDistributed(BatchNormalization()))
mobileNetModel.add(TimeDistributed(MaxPooling2D((2, 2))))
mobileNetModel.add(TimeDistributed(Flatten()))

mobileNetModel.add(LSTM(128))
mobileNetModel.add(Dropout(0.25))

mobileNetModel.add(Dense(128,activation='relu'))
mobileNetModel.add(Dropout(0.25))

mobileNetModel.add(Dense(5, activation='softmax'))


optimiser = optimizers.Adam()
mobileNetModel.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (mobileNetModel.summary())

Model: "sequential_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ time_distributed_24             │ (None, 20, 5, 5, 1024) │     3,228,864 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_25             │ (None, 20, 5, 5, 1024) │         4,096 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_26             │ (None, 20, 2, 2, 1024) │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_27             │ (None, 20, 4096)       │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_16 (LSTM)                  │ (None, 128)            │     2,163,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_24 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_24 (Dense)                │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_25 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_25 (Dense)                │ (None, 5)              │           645 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,413,317 (20.65 MB)

 Trainable params: 2,182,405 (8.33 MB)

 Non-trainable params: 3,230,912 (12.32 MB)

None


In [93]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [94]:
mobileNetModel.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, initial_epoch=0)

Source path =  Project_data/train ; batch size = 30


/var/folders/th/mw8md0ks7rsdgy45pvbb9cgw0000gn/T/ipykernel_1081/2363037342.py:22: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(img_path).astype(np.float32)  # Read the image using imageio


Epoch 1/20
21/23 ━━━━━━━━━━━━━━━━━━━━ 5s 3s/step - categorical_accuracy: 0.3419 - loss: 1.4811

/var/folders/th/mw8md0ks7rsdgy45pvbb9cgw0000gn/T/ipykernel_1081/2363037342.py:59: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(img_path).astype(np.float32)  # Read the image


23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - categorical_accuracy: 0.3509 - loss: 1.4643Source path =  Project_data/val ; batch size = 30

Epoch 1: saving model to model_init_2024-12-2914_16_28.560001/model-00001-1.28836-0.44495-0.92080-0.69000.keras
23/23 ━━━━━━━━━━━━━━━━━━━━ 85s 3s/step - categorical_accuracy: 0.3548 - loss: 1.4570 - val_categorical_accuracy: 0.6900 - val_loss: 0.9208 - learning_rate: 0.0010
Epoch 2/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - categorical_accuracy: 0.7857 - loss: 0.7050
Epoch 2: saving model to model_init_2024-12-2914_16_28.560001/model-00002-0.67111-0.77526-0.65101-0.73000.keras
23/23 ━━━━━━━━━━━━━━━━━━━━ 75s 3s/step - categorical_accuracy: 0.7853 - loss: 0.7036 - val_categorical_accuracy: 0.7300 - val_loss: 0.6510 - learning_rate: 0.0010
Epoch 3/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - categorical_accuracy: 0.8507 - loss: 0.4919
Epoch 3: saving model to model_init_2024-12-2914_16_28.560001/model-00003-0.44830-0.86576-0.60876-0.78000.keras
23/23 ━━━━━━━━

23/23 ━━━━━━━━━━━━━━━━━━━━ 82s 4s/step - categorical_accuracy: 0.9997 - loss: 0.0101 - val_categorical_accuracy: 0.7900 - val_loss: 0.6599 - learning_rate: 8.0000e-06
Epoch 20/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - categorical_accuracy: 0.9999 - loss: 0.0108
Epoch 20: saving model to model_init_2024-12-2914_16_28.560001/model-00020-0.01393-0.99849-0.71122-0.76000.keras
23/23 ━━━━━━━━━━━━━━━━━━━━ 83s 4s/step - categorical_accuracy: 0.9999 - loss: 0.0109 - val_categorical_accuracy: 0.7600 - val_loss: 0.7112 - learning_rate: 8.0000e-06


So we have got the training accuarcy of 99 % and validation accuracy of 76 % in the base model

#### Experiment 10 - Pretrained Mobile net, with learning weights, using GRU

In [44]:
train_doc = np.random.permutation(open('Project_data_Final/train.csv').readlines())
val_doc = np.random.permutation(open('Project_data_Final/val.csv').readlines())
batch_size = 20
num_epochs = 20
print ('# epochs =', num_epochs)

# epochs = 20


In [45]:
from keras.applications import mobilenet

mobilenet_transfer = mobilenet.MobileNet(weights='imagenet', include_top=False)

num_classes = 5

# Input shape: (number of frames, height, width, channels)
input_shape = (18, 120, 120, 3)
        
preTrainedGRU = Sequential()
preTrainedGRU.trainable = True

preTrainedGRU.add(TimeDistributed(mobilenet_transfer,input_shape=(input_shape)))


preTrainedGRU.add(TimeDistributed(BatchNormalization()))
preTrainedGRU.add(TimeDistributed(MaxPooling2D((2, 2))))
preTrainedGRU.add(TimeDistributed(Flatten()))

preTrainedGRU.add(GRU(128))
preTrainedGRU.add(Dropout(0.25))

preTrainedGRU.add(Dense(128,activation='relu'))
preTrainedGRU.add(Dropout(0.25))

preTrainedGRU.add(Dense(num_classes, activation='softmax'))


C:\Users\upujaari\AppData\Local\Temp\ipykernel_27636\4057799317.py:3: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  mobilenet_transfer = mobilenet.MobileNet(weights='imagenet', include_top=False)


In [46]:
# Compile the model with an optimizer
optimiser = optimizers.Adam(learning_rate=0.0001)  # You can experiment with the learning rate
preTrainedGRU.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

# Print the model summary to check the total number of parameters
print(preTrainedGRU.summary())

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ time_distributed_12                  │ (None, 18, 3, 3, 1024)      │       3,228,864 │
│ (TimeDistributed)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_13                  │ (None, 18, 3, 3, 1024)      │           4,096 │
│ (TimeDistributed)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_14                  │ (None, 18, 1, 1, 1024)      │               0 │
│ (TimeDistributed)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_15                  │ (None, 18, 1024)            │               0 │
│ (TimeDistributed)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_3 (GRU)                          │ (None, 128)                 │         443,136 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_6 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 128)                 │          16,512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_7 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 5)                   │             645 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,693,253 (14.09 MB)

 Trainable params: 3,669,317 (14.00 MB)

 Non-trainable params: 23,936 (93.50 KB)

None


In [47]:
train_generator = generator_2(train_path, train_doc, batch_size)
val_generator = generator_2(val_path, val_doc, batch_size)

In [48]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

print('Steps per epoch:', steps_per_epoch)
print('Validation steps:', validation_steps)

Steps per epoch: 34
Validation steps: 5


In [49]:
preTrainedGRU.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, initial_epoch=0)

Source path =  Project_data_Final/train ; batch size = 20
num_batches =  33


C:\Users\upujaari\AppData\Local\Temp\ipykernel_27636\4273982213.py:22: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(img_path).astype(np.float32)  # Read the image using imageio


Epoch 1/20
32/34 ━━━━━━━━━━━━━━━━━━━━ 16s 8s/step - categorical_accuracy: 0.2378 - loss: 1.7694

C:\Users\upujaari\AppData\Local\Temp\ipykernel_27636\4273982213.py:59: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(img_path).astype(np.float32)  # Read the image


num_batches =  33
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 8s/step - categorical_accuracy: 0.2396 - loss: 1.7637Source path =  Project_data_Final/val ; batch size = 20
num_batches =  5
num_batches =  5

Epoch 1: saving model to model_init_2024-12-3014_08_30.722141/model-00001-1.67187-0.26697-1.24460-0.56000.keras
34/34 ━━━━━━━━━━━━━━━━━━━━ 469s 9s/step - categorical_accuracy: 0.2403 - loss: 1.7610 - val_categorical_accuracy: 0.5600 - val_loss: 1.2446 - learning_rate: 1.0000e-04
Epoch 2/20
32/34 ━━━━━━━━━━━━━━━━━━━━ 13s 7s/step - categorical_accuracy: 0.6114 - loss: 1.1226num_batches =  33
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 7s/step - categorical_accuracy: 0.6111 - loss: 1.1202num_batches =  5

Epoch 2: saving model to model_init_2024-12-3014_08_30.722141/model-00002-1.08280-0.60633-0.99587-0.66000.keras
34/34 ━━━━━━━━━━━━━━━━━━━━ 249s 7s/step - categorical_accuracy: 0.6110 - loss: 1.1191 - val_categorical_accuracy: 0.6600 - val_loss: 0.9959 - learning_rate: 1.0000e-04
Epoch 3/20
32/34 ━━━━━━━━━━━━━━━━━